# Based on cb_galex_sedfit.py

### Import relevant packages
### note: need the following --> pip install pysynphot

In [1]:
import numpy as np
import os
import argparse
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.io import ascii
from astropy.io import fits
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
from sedpy import observate
import pysynphot as S

/home/idies/miniconda3/lib/python3.7/site-packages/pysynphot/locations.py:47: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/home/idies/miniconda3/lib/python3.7/site-packages/pysynphot/locations.py:346: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


### List of templates, names of columns, colors

In [2]:
templates = ['Composite1', 'Composite2', 'Composite3', 'AGN1', 'AGN2', 'AGN3', 'AGN4', 'SFG1', 'SFG2', 'SFG3',
             'IR_COLOR1', 'IR_COLOR2', 'IR_COLOR3', 'IR_COLOR4', 'IR_COLOR5', 'IR_COLOR6', 'IR_COLOR7', 'IR_COLOR8']

fits_cols = ['galaxy', 'template_name', 'filter', 'mags', 'wave_eff', 'model_phot', 'model_phot_mags']

colors = {'Composite1': 'silver', 'Composite2': 'rosybrown', 'Composite3': 'darksalmon',
        'AGN1': 'cornflowerblue', 'AGN2': 'blue', 'AGN3': 'slateblue', 'AGN4': 'paleturquoise',
        'SFG1': 'blueviolet', 'SFG2': 'plum', 'SFG3': 'mediumorchid',
        'IR_COLOR1': 'olive', 'IR_COLOR2': 'olivedrab', 'IR_COLOR3': 'yellowgreen', 'IR_COLOR4': 'greenyellow',
        'IR_COLOR5': 'lawngreen', 'IR_COLOR6': 'lightgreen', 'IR_COLOR7': 'darkgreen', 'IR_COLOR8': 'aquamarine'}


### Open table

In [3]:
table = fits.open('hizea_photo_galex_wise_v1.0.fit')

### Set up variables

In [4]:
cols = ['fuv', 'nuv', 'u', 'g', 'r', 'i', 'z', 'w1', 'w2', 'w3', 'w4',
        'fuv_unc', 'nuv_unc', 'u_unc', 'g_unc', 'r_unc', 'i_unc', 'z_unc', 'w1_unc', 'w2_unc', 'w3_unc', 'w4_unc', 'Z']
filt_waves = table[1].data['PHOT_WAVE'][0].byteswap().newbyteorder()*(10**-4)
gals_redshifts = np.array([[i] for i in table[1].data['Z']])
np.array([[i] for i in table[1].data['Z']])
gal_names = table[1].data['SHORT_NAME'].byteswap().newbyteorder()

### Function to convert between f_lambda and f_nu

In [5]:
def flam_to_fnu(flux):
    spec = S.ArraySpectrum(table[1].data['PHOT_WAVE'][0], flux, fluxunits='Flam')
    spec.convert('Fnu')
    return spec.flux

### Make a table with fluxes and errors

In [6]:
flam = table[1].data['FLUX_FLAM'].byteswap().newbyteorder()
fnu = np.array([flam_to_fnu(flammie) for flammie in flam])
flamu = table[1].data['FLUX_FLAM_ERR'].byteswap().newbyteorder()
fnuu = np.array([flam_to_fnu(flammieu) for flammieu in flamu])
flux_w_err = np.concatenate((fnu, fnuu, gals_redshifts), axis=1)
gals_flux = pd.DataFrame(data=flux_w_err,
                        index=gal_names,
                        columns=cols)

Warning, 1 of 11 bins contained negative fluxes; they have been set to zero.


### Make a table with magnitudes and errors

In [7]:
mags = table[1].data['AB_MAG'].byteswap().newbyteorder()
magsu = table[1].data['AB_MAG_ERR'].byteswap().newbyteorder()
mags_w_err = np.concatenate((mags, magsu, gals_redshifts), axis=1)
gals_mag = pd.DataFrame(data=mags_w_err,
                        index=gal_names,
                        columns=cols)

### Functions

In [8]:
# ----- Read the template into a data frame. Name columns something convenient.
def read_template(name):
    temps_path = 'kirkpatrick/'
    temp = pd.read_csv(temps_path+name+'.txt',
                       names=['rest_wavelength','luminosity','DLnu'],
                       skiprows=[0, 1, 2, 3],
                       sep=r'\s{1,}',
                       engine='python')
    return temp

# ----- Function to match a target wavelength's position.
def mask_wave(temp_wavel, target_wave):
    return np.abs(temp_wavel-target_wave) == np.amin(np.abs(temp_wavel-target_wave))

# ----- Designed to take in flux and return mags
def mag(flux):
    mag = -2.5*np.log10(flux)
    return mag

### Fit templates, return column to feed into a new table

In [9]:
def sed_fitting(gal_name, template_name):
    z = gals_mag.loc[gal_name, 'Z']
    template = tempsdf[tempsdf.template_name == template_name]

    # ----- Organizing wavelength and luminosity
    z_temp_wavel = template.rest_wavelength * (1 + z)
    gal_fluxes = gals_flux.loc[gal_name, :][:11].values
    W3_wavelength = filt_waves[9]
    # Figure out where the template lines up with W1
    mask = mask_wave(z_temp_wavel, W3_wavelength)
    # Scale template to match value at W1
    factor = gal_fluxes[9] / float(template.luminosity[mask].values[0])
    luminosity = template.luminosity * factor  # Scale

    # ----- Readying wavelength and flux for sedpy
    wave_aa = np.array(z_temp_wavel[0:-1]) * 1e4
    flux = np.array(luminosity[0:-1])
    fnu = flux * 3631. * 1e-23
    flambda = fnu * 2.998e18 / (wave_aa) ** 2

    # ----- Using sedpy to get wise band photometry based on templates
    filternames = ['wise_w{}'.format(n) for n in ['1', '2', '3', '4']]
    wise_filters = observate.load_filters(filternames)
    model_mags = observate.getSED(wave_aa, flambda, filterlist=wise_filters)
    wave_eff = [f.wave_effective for f in wise_filters]
    model_phot = 10. ** (model_mags / (-2.5))

    rows = pd.DataFrame([[gal_name, template_name, filternames[i],
                          model_mags[i], wave_eff[i], model_phot[i],
                          mag(model_phot[i])] for i in range(len(filternames))],
                        columns=fits_cols)
    return rows

### Plots templates for comparison and calculates lowest chi

In [10]:
def template_comparison(gal_name, template_name):
    z = gals_mag.loc[gal_name, 'Z']
    template = tempsdf[tempsdf.template_name == template_name].reset_index(drop=True)[['rest_wavelength', 'luminosity']]
    z_temp_wavel = template.rest_wavelength * (1 + z)
    gal_fluxes = gals_flux.loc[gal_name, :][:11].values
    W3_wavelength = filt_waves[9]
    # Figure out where the template lines up with W1
    mask = mask_wave(z_temp_wavel, W3_wavelength)
    # Scale template to match value at W1
    factor = gal_fluxes[9]/float(template.luminosity[mask].values[0])
    luminosity = template.luminosity*factor # Scale
    model_phot = sed_fits[(sed_fits.galaxy == gal_name) & (sed_fits.template_name == template_name)].model_phot.array

    gal_unc = gals_flux.iloc[:,11:-1].loc[gal_name].values
    chi = np.sum(np.array([((gal_fluxes[i + 8] - model_phot[i + 1]) / gal_unc[i + 8]) ** 2 for i in range(3)])) / 3

    # Plot
    plot = True

    if plot:
        plot_color = colors[template_name]
        title = gal_name + '-' + template_name

        g = sns.lineplot(x=z_temp_wavel, y=luminosity, color=plot_color, label=template_name, alpha=0.6, ax=ax)
        h = sns.scatterplot(x=filt_waves, y=gal_fluxes, ax=ax, color='blue')
        # ax.scatter(x=np.array(wave_eff)/1e4, y=model_phot, color='red', s=11)
        ax.errorbar(filt_waves, gal_fluxes, yerr=gal_unc, color='blue', ls='none')
        ax.set_ylim([1e-14, 1e-7])
        ax.set_xlim([0.1, 1000.])
        ax.loglog()
        ax.legend()
        ax.set_title(title)
        plt.ioff()
        # plt.savefig(title+'.png')
        # plt.clf()
        # plt.close()

    return (chi)

### Make general plots showing all galaxies (different colors and different filters)

In [11]:
def general_color_plots():
    # Different filters
    fig = plt.figure(figsize=(17, 10))
    ax = fig.add_subplot(1,1,1)
    w2 = sed_fits[sed_fits['filter'] == 'wise_w2'].model_phot_mags.array
    w3 = sed_fits[sed_fits['filter'] == 'wise_w3'].model_phot_mags.array
    w4 = sed_fits[sed_fits['filter'] == 'wise_w4'].model_phot_mags.array
    sns.scatterplot(x=w3 - w4,
                    y=w2 - w3,
                    data=sed_fits.iloc[::4, :].reset_index(), hue='template_name', ax=ax)
    ax.set_ylabel('W3-W4')
    ax.set_xlabel('W2-W3')
    ax.set_title('Color vs color - filters')
    plt.savefig('galex_color_byfilt.png')
    plt.close('all')
    # Different galaxies
    fig = plt.figure(figsize=(17, 10))
    ax = fig.add_subplot(1,1,1)
    w2 = sed_fits[sed_fits['filter'] == 'wise_w2'].model_phot.array
    w3 = sed_fits[sed_fits['filter'] == 'wise_w3'].model_phot.array
    w4 = sed_fits[sed_fits['filter'] == 'wise_w4'].model_phot.array
    sns.scatterplot(x=mag(w3) - mag(w4),
                    y=mag(w2) - mag(w3),
                    data=sed_fits.iloc[::4, :].reset_index(), hue='galaxy', palette='Paired')
    ax.set_ylabel('W3-W4')
    ax.set_xlabel('W2-W3')
    ax.set_title('Color vs color - galaxies')
    plt.savefig('galex_color_bygal.png')
    plt.close('all')


### Making color vs color plots for individual galaxies

In [12]:
def color_plots():
    print(' ---------> Making color plots showing different filters...')
    with PdfPages('galex_sed_fitting_colorplots.pdf') as pdf:
        for galaxy in gal_names:
            print("---Making color plot: ", galaxy)
            fig = plt.figure(figsize=(17, 10))
            ax = fig.add_subplot(1,1,1)
            sedf_g = sed_fits.copy()[sed_fits.galaxy == galaxy].reset_index(drop=True)

            w2 = sedf_g[sedf_g['filter'] == 'wise_w2'].model_phot_mags.array
            w3 = sedf_g[sedf_g['filter'] == 'wise_w3'].model_phot_mags.array
            w4 = sedf_g[sedf_g['filter'] == 'wise_w4'].model_phot_mags.array
            w2_gal = gals_mag['w2'][galaxy]
            w3_gal = gals_mag['w3'][galaxy]
            w4_gal = gals_mag['w4'][galaxy]

            sns.scatterplot(x=w3 - w4, y=w2 - w3,
                            data=sedf_g[sedf_g['filter'] == 'wise_w2'], hue='template_name', ax=ax)

            ax.plot(w3_gal - w4_gal, w2_gal - w3_gal, marker='*', markersize=14, label=galaxy)
            ax.set_title(galaxy)
            ax.set_ylabel('W2-W3')
            ax.set_xlabel('W3-W4')
            pdf.savefig(bbox_inches="tight")
            plt.close('all')
        print('-----Finished color plots pdf')

### Making a table with the templates

In [13]:
print('Reading templates into data frame...')
tempsdf = pd.DataFrame([],columns=['rest_wavelength','luminosity','DLnu'])
for temp_name in templates:
    newdf = read_template(temp_name)
    newdf['template_name'] = [temp_name for i in range(newdf.shape[0])]
    tempsdf = tempsdf.append(newdf)
print('Templates read.')

Reading templates into data frame...
Templates read.


### Generating fits for galaxies

In [14]:
sed_fits = pd.DataFrame([], columns=fits_cols)
print(' ---------> Fitting templates to data...')
for gal in gal_names:
    print("---Fitting ", gal)
    for tem in templates:
        sed_fits = sed_fits.append(sed_fitting(gal, tem))
print('---Finished fitting templates to data.\n')
sed_fits.reset_index(inplace=True, drop=True)

 ---------> Fitting templates to data...
---Fitting  J0106-1023
---Fitting  J0315-0740
---Fitting  J0811+4716
---Fitting  J0824+5032
---Fitting  J0826+4305
---Fitting  J0827+2954
---Fitting  J0901+0314
---Fitting  J0905+5759
---Fitting  J0908+1039
---Fitting  J0933+5614
---Fitting  J0939+4251
---Fitting  J0944+0930
---Fitting  J1036-0102
---Fitting  J1039+4537
---Fitting  J1052+0607
---Fitting  J1052+4104
---Fitting  J1104+5946
---Fitting  J1107+0417
---Fitting  J1125-0145
---Fitting  J1133+0956
---Fitting  J1142+6037
---Fitting  J1205+1818
---Fitting  J1219+0336
---Fitting  J1229+3545
---Fitting  J1232+0723
---Fitting  J1235+6140
---Fitting  J1239+0731
---Fitting  J1244+4140
---Fitting  J1248+0601
---Fitting  J1341-0321
---Fitting  J1359+5137
---Fitting  J1450+4621
---Fitting  J1500+1739
---Fitting  J1506+6131
---Fitting  J1506+5402
---Fitting  J1516+1650
---Fitting  J1558+3957
---Fitting  J1604+3939
---Fitting  J1611+2650
---Fitting  J1613+2834
---Fitting  J1622+3145
---Fitting  J163

### Make color vs color plots

In [15]:
color_plots()
print('-----Generated galex_sed_fitting_colorplots.pdf with color vs color plots.')

 ---------> Making color plots showing different filters...
---Making color plot:  J0106-1023
---Making color plot:  J0315-0740
---Making color plot:  J0811+4716
---Making color plot:  J0824+5032
---Making color plot:  J0826+4305
---Making color plot:  J0827+2954
---Making color plot:  J0901+0314
---Making color plot:  J0905+5759
---Making color plot:  J0908+1039
---Making color plot:  J0933+5614
---Making color plot:  J0939+4251
---Making color plot:  J0944+0930
---Making color plot:  J1036-0102
---Making color plot:  J1039+4537
---Making color plot:  J1052+0607
---Making color plot:  J1052+4104
---Making color plot:  J1104+5946
---Making color plot:  J1107+0417
---Making color plot:  J1125-0145
---Making color plot:  J1133+0956
---Making color plot:  J1142+6037
---Making color plot:  J1205+1818
---Making color plot:  J1219+0336
---Making color plot:  J1229+3545
---Making color plot:  J1232+0723
---Making color plot:  J1235+6140
---Making color plot:  J1239+0731
---Making color plot: 

In [16]:
general_color_plots()
print('----Generated galex_color_byfilt.png and galex_color_bygal.png showing color vs color for all gals.')

----Generated galex_color_byfilt.png and galex_color_bygal.png showing color vs color for all gals.


### Produce a PDF file showing template fits

In [ ]:
# this takes some time to run 10:35am --> 
with PdfPages('galexsed_fitting.pdf') as pdf:
    print('\n ---------> Plotting templates and calculating chi values...\n')
    for gal in gal_names:
        fig = plt.figure(figsize=(17,10))
        ax = fig.add_subplot(1,1,1)
        print("---Fitting ", gal)
        chis = []
        for tem in templates:
            new_chi = template_comparison(gal, tem)
            chis.append(new_chi)
        bestchi_pos = chis.index(min(chis))
        result_string = "Galaxy - " + gal + " - lowest chi tempalte: " + str(templates[bestchi_pos])
        print(result_string)
        plt.text(10, 10**-6.5, result_string, ha='center')
        pdf.savefig(bbox_inches="tight")
        plt.close('all')
    print('Finished!')


 ---------> Plotting templates and calculating chi values...

---Fitting  J0106-1023


/home/idies/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


Galaxy - J0106-1023 - lowest chi tempalte: AGN2
---Fitting  J0315-0740
Galaxy - J0315-0740 - lowest chi tempalte: AGN3
---Fitting  J0811+4716


KeyboardInterrupt: 